In [1]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

import numpy as np
import time
# from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

Using TensorFlow backend.


In [2]:
import keras
print(keras.__version__)

2.1.6


In [3]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

In [4]:
# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

In [5]:
# util function to convert a tensor into a valid image

In [6]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
# this is the placeholder for the input images
input_img = model.input

In [10]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

In [11]:
layer_dict

{'block1_conv1': <keras.layers.convolutional.Conv2D at 0x7f5b78059cc0>,
 'block1_conv2': <keras.layers.convolutional.Conv2D at 0x7f5b394656d8>,
 'block1_pool': <keras.layers.pooling.MaxPooling2D at 0x7f5b39465cc0>,
 'block2_conv1': <keras.layers.convolutional.Conv2D at 0x7f5b393fe7f0>,
 'block2_conv2': <keras.layers.convolutional.Conv2D at 0x7f5b393b81d0>,
 'block2_pool': <keras.layers.pooling.MaxPooling2D at 0x7f5b393c4b70>,
 'block3_conv1': <keras.layers.convolutional.Conv2D at 0x7f5b393d72b0>,
 'block3_conv2': <keras.layers.convolutional.Conv2D at 0x7f5b393e5f28>,
 'block3_conv3': <keras.layers.convolutional.Conv2D at 0x7f5b39388cf8>,
 'block3_pool': <keras.layers.pooling.MaxPooling2D at 0x7f5b3939b860>,
 'block4_conv1': <keras.layers.convolutional.Conv2D at 0x7f5b393aa5f8>,
 'block4_conv2': <keras.layers.convolutional.Conv2D at 0x7f5b3934e978>,
 'block4_conv3': <keras.layers.convolutional.Conv2D at 0x7f5b3935d908>,
 'block4_pool': <keras.layers.pooling.MaxPooling2D at 0x7f5b392ee5f

In [12]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [13]:
kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0
Current loss value: 11.219795
Current loss value: 26.222647
Current loss value: 43.749187
Current loss value: 65.53768
Current loss value: 85.254005
Current loss value: 106.02268
Current loss value: 138.91266
Current loss value: 163.0485
Current loss value: 185.99463
Current loss value: 218.14648
Current loss value: 246.49956
Current loss value: 273.6926
Current loss value: 301.98822
Current loss value: 335.4301
Current loss value: 363.8388
Current loss value: 397.3519
Current loss value: 430.597
Current loss value: 461.59158
Current loss value: 492.50034
Current loss value: 525.4459
Filter 0 processed in 7s
Processing filter 1
Current loss value: 0.040791903
Current loss value: 14.060551
Current loss value: 38.096523
Current loss value: 74.78232
Current loss value: 103.754425
Current loss value: 148.05492
Current loss value: 194.21881
Current loss value: 242.01768
Current loss value: 289.64734
Current loss value: 337.5983
Current loss value: 385.9381
Current loss v

In [14]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

In [15]:
# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

In [16]:
len(kept_filters)

64

In [17]:
# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

In [18]:
# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

In [ ]:
plt.imshow(stitched_filters)